## Preprocessing
---

· Eliminar digits del text 
· Convertir tot el text a minuscula
· Eliminar caracters especials
· Substitueix els espais en blanc continus per un de sol
· Concatena totes les frases amb un espai doble al mig
